In [ ]:
import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams.update({"axes.grid" : True})
font = {'size': 22}
mpl.rc('font', **font)

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/test_NJNpi_GEVP_noPost/templateFiles/templateMomList.h5'
with h5py.File(path) as f:
    momList=f['B'][()]
    for i,mom in enumerate(momList):
        print(i,mom)

In [ ]:
path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/test_NJNpi_GEVP_noPost/avg-merge_backup/'
pi2Str='pi2=0_0_1'; mom=13; ab=0

data={}
for pat in ['B','W','Z']:
    with h5py.File(path+pat+'.h5') as f:
        for cont in f['0000/12'][pi2Str].keys():
            if cont.endswith('mvec'):
                continue
            data[cont]=np.array([f[cfg]['12'][pi2Str][cont][:,mom,:,ab] for cfg in f.keys()])
            print(cont)

In [ ]:
tfList=[10,12,14]

def run(conts,Gc,preFactor=1):
    fig,axs=yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)

    def tFunc(dat):
        t=np.mean(dat,axis=0)
        return t

    for cont in conts:
        dat=np.array([np.array([np.real(preFactor*data[cont+'_deltat_'+str(tf)][cfg,1:tf,Gc]) for tf in tfList],dtype=object) for cfg in range(300)])
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            irow=0; icol=0
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white')

for num in range(3,12+1):
    run(['B'+str(num)],4,-1j)

In [ ]:
for num in range(5,24+1):
    run(['W'+str(num)],4,-1j)

In [ ]:
for num in range(5,14+1):
    run(['Z'+str(num)],5,1)

In [ ]:
for num in range(3,12+1):
    run(['B'+str(num)],5,1)